## 1. 추천 시스템
- 사용자 (user)에게 관련된 아이템 (item)을 추천해 주는 것
- **범주형 데이터를 다룬다.**
액션물, 로맨스물, 스릴러물, 한드, 미드, 일드, 영드 등의 영화 item 데이터와 A, B, C 같은 user 데이터를 취급했습니다. 이러한 데이터는 연속적(continuous)이지 않고 이산적(discrete)입니다. 이를 범주형(categorical) 데이터라고 합니다.
- **(숫자 벡터로 변환한 뒤) 유사도를 계산한다.**
범주형 데이터들을 좌표에 나타내었는데, 좌표에 나타내기 위해서는 숫자로 이루어진 벡터(numerical vector)로 변환해야 합니다. 그리고 그 거리를 계산하여 유사도를 계산합니다.

- 범주형, 이산적인 데이터를 숫자 벡터로 변환합니다.
- 이 숫자 벡터의 유사도를 계산하여 유사도가 가까운 (혹은 높은) 제품을 추천해 줍니다.

## 2. 코사인 유사도
- 코사인 유사도는 두 벡터 간의 코사인 값을 이용해 두 벡터의 유사도를 계산합니다.
- 코사인 유사도는 두 벡터의 방향이 이루는 각에 코사인을 취해 구합니다. 따라서, 두 벡터의 방향이 완전히 동일한 경우는 1, 90°의 각을 이루면 0, 반대 방향, 즉 각이 180°이면 -1의 값을 갖게 됩니다. 따라서 코사인 유사도는 -1 ~ 1사이의 값이고, 1에 가까울 수록 유사도가 높다고 할 수 있습니다.

${\displaystyle {\text{cosine similarity}}=\cos(\theta )={\mathbf {A} \cdot \mathbf {B} \over \|\mathbf {A} \|\|\mathbf {B} \|}={\frac {\sum_{i=1}^n {A_{i}B_{i}}}{{\sqrt {\sum_{i=1}^n{A_{i}^{2}}}}{\sqrt {\sum_{i=1}^n{B_{i}^{2}}}}}},}$

### 2-1. NumPy 활용

In [7]:
import numpy as np

t1 = np.array([1,1,1])
t2 = np.array([2,0,1])

In [10]:
# 코사인 유사도 구하는 함수
from numpy import dot
from numpy.linalg import norm
def cos_sim(A,B):
    return dot(A,B)/(norm(A)*norm(B))

In [11]:
cos_sim(t1,t2)

0.7745966692414834

### 2-2. 사이킷런 활용

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
# 해당 모듈은 입력값으로 2차원 배열을 받기 때문에, 2차원 배열로 정의해준다.
t1 = np.array([[1,1,1]])
t2 = np.array([[2,0,1]])
cosine_similarity(t1,t2)

array([[0.77459667]])

기타 다른 유사도 계산법

코사인 유사도 이외에도 유클리드 거리, 자카드 유사도, 피어슨 상관계수 등의 다른 방법으로 유사도를 계산할 수 있습니다.

## 3. 추천시스템의 종류
- 콘텐츠 기반 필터링(Content Based Filtering)

- 협업 필터링(Collaborative Filtering)

    - 사용자 기반
    - 아이템 기반
    - 잠재요인 협업 필터링 (latent factor collaborative filtering) → 행렬 인수분해(matrix factorization)
- Deep Learning 적용 or Hybrid 방식

### 3-1. 콘텐츠 기반 필터링 
- 어떤 사람이 한 영화를 좋아했다면, 비슷한 콘텐츠의 아이템을 추천하는 방식
- 콘텐츠 기반은 순수하게 콘텐츠의 내용만을 비교해서 추천하는 방식
- 특성 (Feature)의 유사성에 의존

In [16]:
# 필요한 모듈 임포트
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

 #로드 데이터
movie_dataset.csv 
wget https://aiffelstaticprd.blob.core.windows.net/media/documents/movie_dataset.csv
mv movie_dataset.csv  ~/aiffel/movie_recommendation

In [17]:
# 로드 데이터
import os
csv_path = os.getenv('HOME')+'/workplace/aiffel/Fundamentals/26.movie_recommendation/movie_dataset.csv'
df = pd.read_csv(csv_path)
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [18]:
# 특성 선택
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [19]:
features = ['keywords','cast','genres','director']
features

['keywords', 'cast', 'genres', 'director']

In [20]:
def combine_features(row):
    return row['keywords']+""+row['cast']+""+row['genres']+""+row['director']

combine_features(df[:5])

0    culture clash future space war space colony so...
1    ocean drug abuse exotic island east india trad...
2    spy based on novel secret agent sequel mi6Dani...
3    dc comics crime fighter terrorist secret ident...
4    based on novel mars medallion space travel pri...
dtype: object

In [22]:
for feature in features:
    df[feature] = df[feature].fillna('')
    
df["combined_features"] = df.apply(combine_features,axis=1)
df["combined_features"]

0       culture clash future space war space colony so...
1       ocean drug abuse exotic island east india trad...
2       spy based on novel secret agent sequel mi6Dani...
3       dc comics crime fighter terrorist secret ident...
4       based on novel mars medallion space travel pri...
                              ...                        
4798    united states\u2013mexico barrier legs arms pa...
4799    Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...
4800    date love at first sight narration investigati...
4801    Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...
4802    obsession camcorder crush dream girlDrew Barry...
Name: combined_features, Length: 4803, dtype: object

In [23]:
# 벡터화하고, 코사인 유사도 계산
cv = CountVectorizer()
count_matrix = cv.fit_transform(df['combined_features'])
print(type(count_matrix)) # CSR(Compressed Sparse Row) Matrix
print(count_matrix.shape)
print(count_matrix)

<class 'scipy.sparse.csr.csr_matrix'>
(4803, 23553)
  (0, 4909)	1
  (0, 3863)	1
  (0, 8447)	1
  (0, 19658)	2
  (0, 22508)	1
  (0, 4051)	1
  (0, 19544)	1
  (0, 23196)	1
  (0, 23515)	1
  (0, 18409)	1
  (0, 19275)	1
  (0, 22695)	1
  (0, 19962)	1
  (0, 12267)	1
  (0, 14125)	1
  (0, 17742)	1
  (0, 289)	1
  (0, 7364)	1
  (0, 18733)	1
  (0, 7702)	1
  (0, 3073)	1
  (1, 7364)	1
  (1, 15519)	1
  (1, 6315)	1
  (1, 103)	1
  :	:
  (4801, 2103)	1
  (4801, 5053)	1
  (4801, 16053)	1
  (4801, 9825)	1
  (4801, 552)	1
  (4801, 6744)	1
  (4801, 23488)	1
  (4801, 4547)	1
  (4801, 18242)	1
  (4801, 19149)	1
  (4801, 10434)	1
  (4802, 15509)	1
  (4802, 6267)	1
  (4802, 2669)	1
  (4802, 6894)	1
  (4802, 4462)	1
  (4802, 1689)	1
  (4802, 17684)	1
  (4802, 9192)	1
  (4802, 3063)	1
  (4802, 4874)	1
  (4802, 7555)	1
  (4802, 8864)	1
  (4802, 9909)	2
  (4802, 6397)	1


CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보로만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조입니다.

(0, 3115) 1 : 1번째 row는 3116번째 단어가 1번 출현한다는 뜻입니다. 이 데이터셋에는 총 14845개의 단어가 존재하는데, 이 단어들을 범주형으로 보고 그 단어의 출현 빈도만을 표시한 Matrix가 매우 Sparse하기 때문에 공간을 절약할 수 있는 형태로 표현한 것입니다.

In [24]:
# 코사인 유사도를 표현하는 4803X4803 의 cosine_similarity matrix를 구해 봅시다.
cosine_sim = cosine_similarity(count_matrix)
print(cosine_sim)
print(cosine_sim.shape)

[[1.         0.03928371 0.04564355 ... 0.         0.         0.        ]
 [0.03928371 1.         0.         ... 0.0410305  0.         0.        ]
 [0.04564355 0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.0410305  0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
(4803, 4803)


In [25]:
# 코사인 유사도로 계산된 가장 비슷한 영화를 3편을 선별하여 추천
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

movie_user_likes = "Avatar"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index]))

sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

i=0
print(movie_user_likes+"와 비슷한 영화 3편은 "+"\n")
for item in sorted_similar_movies:
    print(get_title_from_index(item[0]))
    i=i+1
    if i==3:
        break

Avatar와 비슷한 영화 3편은 

Guardians of the Galaxy
Alien
Aliens


### 3-2. 협업 필터링
과거의 사용자 행동 양식(User Behavior) 데이터를 기반으로 추천하는 방식입니다. 여기서 다루는 학습용 데이터 자체에서 사용자 행동양식을 나타내기 어렵지만, 잠재요인 기법을 활용하면 행동양식을 어느 정도 데이터로 나타낼 수 있습니다.

 영화 추천을 예시로 들어보겠습니다. 아래 표를 살펴볼까요. 사용자들의 정보는 user_id에 저장되어 있습니다. 그리고 영화 정보는 item_id입니다. 사용자는 영화를 보고 평점 (rating)을 매겼습니다. 그 평점을 매긴 시각이 timestamp 칼럼에 기록되어있습니다.

위 데이터를 사용자와 아이템 간 interaction matrix로 변환합니다. 해당 행렬의 데이터로 평점을 넣으면 아래와 같은 형태가 되겠죠? 우리는 이를 평점행렬이라고 부르기도 합니다.

이러한 행렬을 실제 데이터로 만든다면 굉장히 희소(sparse)한 행렬이 만들어집니다. 유튜브나 넷플릭스에는 몇 억 개의 동영상이 있고, 몇 억 명의 사용자가 모든 동영상을 다 봤을리 만무합니다. 따라서 대부분 평점에 대한 데이터는 0입니다.


#### 3-2-1. 사용자 기반과 아이템 기반

사용자 기반과 아이템 기반 그리고 잠재요인(latent factor) 방식이 있다고 했는데요. 
- 사용자 기반과 아이템 기반은 유사도를 계산하는 방식이고 
- 잠재요인은 행렬 인수분해(matrix factorization)를 이용해 잠재요인을 분석합니다. 넷플릭스의 추천 시스템에 잠재요인 분석을 사용하여 최근에는 잠재요인을 분석하는 사례가 많아지고 있는 추세입니다. 
- 즉, 위에서 설명한 평점행렬로 변환한 후, 평점행렬의 유사도를 계산하여 추천하는 방식은 사용자 기반과 아이템 기반 방식입니다. 그리고 이 평점행렬을 분해하여 더 많은 정보들을 고려하는 방식이 잠재요인 필터링입니다.

#### 사용자 기반
- 최근접 이웃 협업 필터링은 다시 사용자 기반과 아이템 기반으로 나뉜다.
- 동일한 제품에 대해 평점을 매긴 데이터를 분석하여 추천을 하는 방식
- **당신과 비슷한 고객들이 다음 상품을 구매했습니다.**

#### 아이템 기반
- 아이템 간의 유사도를 측정하여 해당 아이템을 추천하는 방식
- 일반적으로 사용자 기반보다는 아이템 기반 방식이 정확도가 더 높다.
- **이 상품을 선택한 다른 고객들은 다음 상품을 구매했습니다.**

**결론**
- 사용자 기반: 당신과 비슷한 고객들이 다음 상품을 구매했습니다.

- 아이템 기반: 이 상품을 선택한 다른 고객들은 다음 상품을 구매했습니다.

#### 3-2-2. 잠재요인 헙업 필터링 - 행렬 인수분해 (matrix factorization)
- 행렬 인수분해를 통해 잠재요인 (latent factor)을 분석한다. 
- 행렬 인수분해
    - SVD(Singular Vector Decomposition)
    - ALS(Alternating Least Squares)
    - NMF(Non-Negative Factorization)
    - 분해 행렬을 사용하면 원하는 답을 더 쉽게 찾을 수 있으며, 추천 알고리즘에서 파라미터 수가 줄어든다.

**SVD**
- 특잇값 분해
- 정보복원을 위해 사용됨
    - 특이값 분해는 분해되는 과정보다는 분해된 행렬을 다시 조합하는 과정에서 그 응용력이 빛을 발한다.
    - 기존의 U,Σ,VTU,Σ,VT로 분해되어 있던 AA행렬을 특이값 p개만을 이용해 A’라는 행렬로 ‘부분 복원’ 시킨 할 수 있다. 위에서 말했던 것 특이값의 크기에 따라 A의 정보량이 결정되기 때문에 값이 큰 몇 개의 특이값들을 가지고도 충분히 유용한 정보를 유지할 수 있다.
    - 최대한 중요한 정보들만 부분 복원해서 사용하면 사진의 용량은 줄어들지만 여전히 사진이 보여주고자 하는 내용은 살릴 수 있을 것이다.

In [2]:
import numpy as np
from numpy.linalg import svd

np.random.seed(30)
A = np.random.randint(0, 100, size=(4,4))
A

array([[37, 37, 45, 45],
       [12, 23,  2, 53],
       [17, 46,  3, 41],
       [ 7, 65, 49, 45]])

In [3]:
svd(A)

(array([[-0.54937068, -0.2803037 , -0.76767503, -0.1740596 ],
        [-0.3581157 ,  0.69569442, -0.13554741,  0.60777407],
        [-0.41727183,  0.47142296,  0.28991733, -0.72082768],
        [-0.6291496 , -0.46389601,  0.55520257,  0.28411509]]),
 array([142.88131188,  39.87683209,  28.97701433,  14.97002405]),
 array([[-0.25280963, -0.62046326, -0.4025583 , -0.6237463 ],
        [ 0.06881225, -0.07117038, -0.8159854 ,  0.56953268],
        [-0.73215039,  0.61782756, -0.23266002, -0.16767299],
        [-0.62873522, -0.47775436,  0.34348792,  0.50838848]]))

결과값으로 행렬 U와 행렬 Σ, 행렬 V의 전치행렬이 생성됩니다. 우리는 이 값들을 unpacking해서 각각 변수명에 할당해 줍니다. 

In [4]:
U, Sigma, VT = svd(A)

print('U matrix: {}\n'.format(U.shape),U)
print('Sigma: {}\n'.format(Sigma.shape),Sigma)
print('V Transpose matrix: {}\n'.format(VT.shape),VT)

U matrix: (4, 4)
 [[-0.54937068 -0.2803037  -0.76767503 -0.1740596 ]
 [-0.3581157   0.69569442 -0.13554741  0.60777407]
 [-0.41727183  0.47142296  0.28991733 -0.72082768]
 [-0.6291496  -0.46389601  0.55520257  0.28411509]]
Sigma: (4,)
 [142.88131188  39.87683209  28.97701433  14.97002405]
V Transpose matrix: (4, 4)
 [[-0.25280963 -0.62046326 -0.4025583  -0.6237463 ]
 [ 0.06881225 -0.07117038 -0.8159854   0.56953268]
 [-0.73215039  0.61782756 -0.23266002 -0.16767299]
 [-0.62873522 -0.47775436  0.34348792  0.50838848]]


잘 분해가 되었군요. 이제 다시 복원해 보겠습니다. 복원을 위해서는 U, Σ, VT를 내적합니다. 한 가지 유의할 사항은 Σ는 1차원 이므로 0을 포함한 대각 행렬로 변환한 뒤 내적을 해주어야합니다.

In [5]:
Sigma_mat = np.diag(Sigma)

A_ = np.dot(np.dot(U, Sigma_mat), VT)
A_

array([[37., 37., 45., 45.],
       [12., 23.,  2., 53.],
       [17., 46.,  3., 41.],
       [ 7., 65., 49., 45.]])

**복원된 A_와 원본 A는 같다.**

#### Truncated SVD
추천 시스템에서의 행렬 인수분해는 SVD 중에서도 Truncated SVD를 사용합니다.

Truncated SVD는 SVD의 일종으로 우리말로 번역하면 잘린 SVD라고합니다. 다른 말로 LSA(Latent semantic analysis), 잠재 의미 분석 이라고 번역할 수 있습니다. Truncated SVD를 이용해 분해한 뒤 복원하면 SVD 처럼 완벽히 같은 행렬이 나오지 않습니다. 그 이유는 Truncated SVD는 차원을 축소한 다음 행렬을 분해하기 때문입니다.

#### 3-2-3. 행렬 인수분해와 잠재요인 협업 필터링
- 사용자가 평점을 매기는 요인을 그냥 "잠재요인"으로 취급한 뒤 그걸 SVD기법을 이용해 분해한 뒤 다시 합치는 방법으로 영화에 평점을 매긴 이유를 벡터화하여 이를 기반으로 추천합니다. 이 기법은 넷플릭스나 왓챠, 유튜브 같은 대 기업에서 사용하여 그 효과를 입증해 내었고 이후 많은 기업들이 이 기법을 채택하여 사용하고 있습니다.

- 이렇듯 협업 필터링을 이용하면 사용자가 아이템에 대해 평점을 매긴 평점행렬을 행렬 인수분해(Matrix Factorization)를 통해 잠재요인을 분석한 뒤 유사도를 계산할 수도 있고 사용자의 평점도 예측할 수 있습니다.

## 4. 실제 추천 시스템
사용자의 구매 여부와 평점 데이터 뿐만 아니라 얼마나 오랜 시간 동안 시청(혹은 해당 웹 사이트에 머물렀는지), 어떤 사이트에서 유입이 되었는지, 그리고 시청한 뒤 구매로 이어지기까지의 시간 등 우리의 족적들을 다 분석합니다. 이를 전문 용어로 Digital Footprint(디지털 발자국), Digital Shadow(디지털 그림자)라고 해요.

그리고 이중에서 가장 중요한 지표가 바로 클릭률입니다. 전문 용어로는 CTR(Click Through Rate)입니다. CTR은 마케팅에서도 중요한 지표로 작용하는 용어이기도합니다.

이러한 데이터들을 모아 추천을 한 뒤, 해당 아이템이 적절한 추천인지 여부를 평가하는 것 역시 중요한 일입니다. 추천한 제품이 구매로 이어졌는지를 통해 추천에 성공했는지를 평가하기도 하고 모델 단계에서 평가하기도 합니다.

추천 시스템은 굉장히 큰 시스템입니다. 데이터를 기반으로 사용자에게 적절한 제품을 추천한다는 것 그리고 그것이 구매로 이어지는것은 매출과 직결되는 문제이기도 합니다. 좋은 추천 시스템을 만들기 위해서는 어떤 데이터를 쓸지 많은 고민이 필요합니다. 사용자와 연관성이 있고, 구매와 직결되는 각종 데이터를 수집하고 정렬(sorting)하여 다시 순위(ranking)를 매긴 다음 평가하는 작업을 반복해가며 적합한 데이터와 추천 시스템을 만들어냅니다.

https://www.youtube.com/watch?time_continue=1&v=ZspR5PZemcs&feature=emb_logo